In [15]:
from PIL import Image
import face_recognition
import os
import cv2
import numpy as np

In [16]:
# Creating function to exract frames from a video, in order to train the model
def extract_frames(video_path, dest_path, frame_rate=1):
    """
    Extracts frames from a video.
    
    :param video_path: Path to the video file.
    :param dest_path: Directory where extracted frames will be saved.
    :param frame_rate: Extract every 'frame_rate' frames. 1 means every frame, 2 means every second frame, etc.
    """
    if not os.path.exists(dest_path):
        os.makedirs(dest_path)

    vidcap = cv2.VideoCapture(video_path)
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    count = 0

    while vidcap.isOpened():
        success, image = vidcap.read()
        if not success:
            break
        if count % frame_rate == 0:
            cv2.imwrite(os.path.join(dest_path, f"frame_{count}.jpg"), image)
        count += 1

    vidcap.release()
    cv2.destroyAllWindows()

In [17]:
# Base directory for training videos
training_videos_base_dir = '/root/computer_vision/face-recognition/training'

# Base directory where extracted frames will be saved
extracted_frames_base_dir = '/root/computer_vision/face-recognition/training'

# Function to process each individual's videos
def process_individual_videos(individual_dir, individual_name):
    individual_videos_path = os.path.join(training_videos_base_dir, individual_dir)
    individual_frames_dest_path = os.path.join(extracted_frames_base_dir, individual_name)
    
    for video_file in os.listdir(individual_videos_path):
        if video_file.endswith('.mp4'):
            video_path = os.path.join(individual_videos_path, video_file)
            dest_path = os.path.join(individual_frames_dest_path, video_file.split('.')[0])  # Create a folder for each video
            print(f"Extracting frames from {video_path} to {dest_path}")
            extract_frames(video_path, dest_path, frame_rate=5)

# Loop through each individual's directory and process their videos
for individual_dir in os.listdir(training_videos_base_dir):
    if os.path.isdir(os.path.join(training_videos_base_dir, individual_dir)):
        process_individual_videos(individual_dir, individual_dir)


Extracting frames from /root/computer_vision/face-recognition/training/claytonkershaw/kershaw2.mp4 to /root/computer_vision/face-recognition/training/claytonkershaw/kershaw2
Extracting frames from /root/computer_vision/face-recognition/training/claytonkershaw/kershaw1.mp4 to /root/computer_vision/face-recognition/training/claytonkershaw/kershaw1
Extracting frames from /root/computer_vision/face-recognition/training/willsmith/willsmith2.mp4 to /root/computer_vision/face-recognition/training/willsmith/willsmith2
Extracting frames from /root/computer_vision/face-recognition/training/willsmith/willsmith1.mp4 to /root/computer_vision/face-recognition/training/willsmith/willsmith1
Extracting frames from /root/computer_vision/face-recognition/training/willsmith/willsmith3.mp4 to /root/computer_vision/face-recognition/training/willsmith/willsmith3


In [21]:
individuals_dir = "/root/computer_vision/face-recognition/training/"
known_face_encodings = []
known_face_names = []

# Function to process images in each subfolder and generate encodings
def process_images_for_individual(individual_frames_path, individual_name):
    # Loop through each subfolder in the individual's directory
    for subfolder in os.listdir(individual_frames_path):
        subfolder_path = os.path.join(individual_frames_path, subfolder)
        if os.path.isdir(subfolder_path):
            # Process each image file in the subfolder
            for image_file in os.listdir(subfolder_path):
                if image_file.endswith('.jpg'):
                    image_path = os.path.join(subfolder_path, image_file)
                    image = face_recognition.load_image_file(image_path)
                    encodings = face_recognition.face_encodings(image)
                    if encodings:
                        # Assuming one face per training image for simplicity
                        known_face_encodings.append(encodings[0])
                        known_face_names.append(individual_name)

# Loop through each individual's directory and process their subfolders
for individual_dir in os.listdir(extracted_frames_base_dir):
    individual_path = os.path.join(extracted_frames_base_dir, individual_dir)
    if os.path.isdir(individual_path):
        process_images_for_individual(individual_path, individual_dir)


In [22]:
print("Initializing...")

video_path = '/root/computer_vision/face-recognition/testing/claytonkershaw/kershaw2.mp4'
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('face_rec_output.mp4',
                      fourcc, 30.0, (frame_width, frame_height))

print(f"Loaded {len(known_face_encodings)} face encodings.")

# Initialize lists to store face locations, encodings, and names
face_locations = []
face_encodings = []
face_names = []
# Initialize variable to control frame processing
process_this_frame = True

print("Starting video processing...")

# Loop over frames from the video file stream
while cap.isOpened():
    # Read the next frame from the file
    ret, frame = cap.read()
    # If the frame was not grabbed, then we have reached the end of the stream
    if not ret:
        print("Reached end of video.")
        break

    # Only process every other frame of video to save time
    if process_this_frame:
        print("Processing frame...")
        # Resize frame of video for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
        small_frame = frame 

        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(small_frame)
        print(f"Found {len(face_locations)} face(s) in this frame.")

        face_encodings = face_recognition.face_encodings(
            small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(
                known_face_encodings, face_encoding, tolerance=0.6)
            name = "Unknown"

            # Use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(
                known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)
        print(f"Identified faces: {face_names}")

    # Switch to not process the next frame
    process_this_frame = not process_this_frame

    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35),
                      (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6),
                    font, 1.0, (255, 255, 255), 1)

    # Write the resulting image to the output video file
    out.write(frame)

print("Releasing video objects...")
# Release the file pointers
out.release()
cap.release()

Initializing...
Loaded 606 face encodings.
Starting video processing...
Processing frame...
Found 0 face(s) in this frame.
Identified faces: []
Processing frame...
Found 0 face(s) in this frame.
Identified faces: []
Processing frame...
Found 0 face(s) in this frame.
Identified faces: []
Processing frame...
Found 0 face(s) in this frame.
Identified faces: []
Processing frame...
Found 0 face(s) in this frame.
Identified faces: []
Processing frame...
Found 0 face(s) in this frame.
Identified faces: []
Processing frame...
Found 0 face(s) in this frame.
Identified faces: []
Processing frame...
Found 0 face(s) in this frame.
Identified faces: []
Processing frame...
Found 0 face(s) in this frame.
Identified faces: []
Processing frame...
Found 0 face(s) in this frame.
Identified faces: []
Processing frame...
Found 0 face(s) in this frame.
Identified faces: []
Processing frame...
Found 0 face(s) in this frame.
Identified faces: []
Processing frame...
Found 0 face(s) in this frame.
Identified fa

In [23]:
print("Initializing...")

video_path = '/root/computer_vision/face-recognition/testing/willsmith/willsmithtest1.mp4'
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('face_rec_output_2.mp4',
                      fourcc, 30.0, (frame_width, frame_height))

print(f"Loaded {len(known_face_encodings)} face encodings.")

# Initialize lists to store face locations, encodings, and names
face_locations = []
face_encodings = []
face_names = []
# Initialize variable to control frame processing
process_this_frame = True

print("Starting video processing...")

# Loop over frames from the video file stream
while cap.isOpened():
    # Read the next frame from the file
    ret, frame = cap.read()
    # If the frame was not grabbed, then we have reached the end of the stream
    if not ret:
        print("Reached end of video.")
        break

    # Only process every other frame of video to save time
    if process_this_frame:
        print("Processing frame...")
        # Resize frame of video for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
        small_frame = frame 

        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(small_frame)
        print(f"Found {len(face_locations)} face(s) in this frame.")

        face_encodings = face_recognition.face_encodings(
            small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(
                known_face_encodings, face_encoding, tolerance=0.6)
            name = "Unknown"

            # Use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(
                known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)
        print(f"Identified faces: {face_names}")

    # Switch to not process the next frame
    process_this_frame = not process_this_frame

    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35),
                      (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6),
                    font, 1.0, (255, 255, 255), 1)

    # Write the resulting image to the output video file
    out.write(frame)

print("Releasing video objects...")
# Release the file pointers
out.release()
cap.release()

Initializing...
Loaded 606 face encodings.
Starting video processing...
Processing frame...
Found 1 face(s) in this frame.
Identified faces: ['willsmith']
Processing frame...
Found 1 face(s) in this frame.
Identified faces: ['willsmith']
Processing frame...
Found 1 face(s) in this frame.
Identified faces: ['willsmith']
Processing frame...
Found 1 face(s) in this frame.
Identified faces: ['willsmith']
Processing frame...
Found 1 face(s) in this frame.
Identified faces: ['willsmith']
Processing frame...
Found 1 face(s) in this frame.
Identified faces: ['willsmith']
Processing frame...
Found 1 face(s) in this frame.
Identified faces: ['willsmith']
Processing frame...
Found 1 face(s) in this frame.
Identified faces: ['willsmith']
Processing frame...
Found 1 face(s) in this frame.
Identified faces: ['willsmith']
Processing frame...
Found 1 face(s) in this frame.
Identified faces: ['willsmith']
Processing frame...
Found 1 face(s) in this frame.
Identified faces: ['willsmith']
Processing fram